**<center><font size=6>CS490: Machine Learning<br>Homework 3</font></center>**

<hr>

# Part 1: Linear Regression (60 points)

For this problem we will fit linear regression models that minimize the *mean squared error* (MSE).

## Preparing the Data

Load the `data/curve80.txt` data set, and split it into 75% / 25% training/test. The first column is the scalar feature value x; the second column data[:,1] is the target value y for each example. For consistency in our results, ***do not reorder (shuffle)*** the data (they’re already in a random
order), and use the first 75% of the data for training and the rest for testing.

In [1]:
# TODO: Load your data into a matrix called data


# Set up training and test sets
X = data[:,0]
X = np.atleast_2d(X).T # code expects shape (M,N) so make sure it's 2-dimensional
Y = data[:,1]
Xtrain,Xtest,Ytrain,Ytest = ml.splitData(X,Y,0.75) # split data set 75/25

**Print the shapes of these objects. <span style="color: deepskyblue;">(5 points)</span>**

In [3]:
# TODO: Print the shapes of the training and validation data




## Linear Regression Learning
Use the provided `linearRegress` class in `mltools` to create a linear regression predictor of y given x. You can plot the resulting function by simply evaluating the model at a large number of x values and plotting the resulting ys:

In [ ]:
lr = ml.linear.linearRegress( Xtrain, Ytrain ) # create and train model
xs = np.linspace(0,10,200) # densely sample possible x-values
xs = xs[:,np.newaxis] # force "xs" to be an Mx1 matrix (expected by our code) 
ys = lr.predict( xs ) # make predictions at xs

**Plot the training data points along with your prediction function in a single plot. <span style="color: deepskyblue;">(5 points)</span>**

In [ ]:
# TODO: plot of training data and prediction function




**Print the linear regression coefficients (`lr.theta`) and verify that they match your plot. <span style="color: deepskyblue;">(5 points)</span>**

In [ ]:
# TODO: print out the regression parameters




**What is the mean squared error of the predictions on the training and test data? Fill in the function below and use to it compute the MSE.  <span style="color: deepskyblue;">(15 points)</span>**

In [ ]:
# Function to compute the MSE
def MSE(y_true, y_hat):
    
    
    

# Use this function to compute MSE
ys = lr.predict(Xtrain)
print("MSE (Train) = ", MSE(Ytrain,ys))

ys = lr.predict(Xtest)
print("MSE (Test)  = ", MSE(Ytest,ys))

You can check your previous answer by using the built-in `mse` function in `mltools`.

In [ ]:
print("MSE (Train) = ", lr.mse(Xtrain,Ytrain))
print("MSE (Test)  = ", lr.mse(Xtest,Ytest))

<hr>

## Regression with Higher Degree Polynomials
Try fitting y = f(x) using a polynomial function f(x) of increasing order. Do this by the trick of adding additional polynomial features before constructing and training the linear regression object. You can do this easily yourself; you can add a quadratic feature of `Xtrain` with:

In [ ]:
Xtrain2 = np.zeros( (Xtrain.shape[0],2) ) # create Mx2 array to store features 
Xtrain2[:,0] = Xtrain[:,0]                # place original "x" feature as X1
Xtrain2[:,1] = Xtrain[:,0]**2             # place "x^2" feature as X2
# Now, Xtrain2 has two features about each data point: "x" and "x^2"

(You can also add the all-ones constant feature in a similar way, but this is currently done automatically within the learner’s train function.) A function `ml.transforms.fpoly` is also provided to more easily create such features. Note, though, that the resulting features may include extremely large values; if x ≈ 10, then x^10 is extremely large. For this reason (as is often the case with features on very different scales) it’s a good idea to **rescale the features**, that is to make sure that the range (the min and the max) and the variance (the spread) are similar across all the features. You can do this manually by computing multipliers for each variable to regularize their values (see https://en.wikipedia.org/wiki/Feature_scaling for some methods) or use the provided function in mltools:

In [ ]:
# Create polynomial features up to "degree"; don't create constant feature
# (the linear regression learner will add the constant feature automatically) 
XtrainP = ml.transforms.fpoly(Xtrain, degree, bias=False)

# Rescale the data matrix so that the features have similar ranges / variance
XtrainP,params = ml.transforms.rescale(XtrainP)

# "params" returns the transformation parameters (shift & scale)
# Then we can train the model on the scaled feature matrix:
lr = ml.linear.linearRegress( XtrainP, Ytrain ) # create and train model

# Now, apply the same polynomial expansion & scaling transformation to Xtest:
XtestP,_ = ml.transforms.rescale( ml.transforms.fpoly(Xtest,degree,false), params)

The transformations used to create features of the training data may depend on properties of that data (such as rescaling the data to have mean zero and variance one). For our learned predictions to be consistent, we need to apply the same transform to new test data, so that it will be represented consistently with the training data. “Feature transform” functions like `rescale` are written to output their parameters (here `params = (mu,sig)`,a tuple containing the mean and standard deviation used to shift and scale the data) so that they can be reused on subsequent data. When evaluating a polynomial regression model, be sure that the same rescaling and polynomial expansion is applied to both the training and test data.

**Train polynomial regression models of degree d = 1, 3, 5, 7, 10, 15, 18, and:**

**(a) For each model, plot the learned prediction function f(x).  <span style="color: deepskyblue;">(15 points)</span>**

**(b) Plot the training and test errors on a log scale ( semilogy ) as a function of the model degree.  <span style="color: deepskyblue;">(10 points)</span>**

**(c) What polynomial degree do you recommend?  <span style="color: deepskyblue;">(5 points)</span>**

When plotting prediction functions in part (a), you should set all plots to have the same vertical axis limits as the d = 1 regression model. Otherwise, high-degree polynomials may appear flat due to taking on extremely large values for a subset of inputs. Here is some example code:

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 8)) # Create axes for single subplot 
ax.plot(...) # Plot polynomial regression of desired degree
ax.set_ylim(..., ...) # Set the minimum and maximum limits
plt.show()

In [ ]:
# TODO: Train and plot polynomials for d = 1, 3, 5, 7, 10, 15, 18




<hr>

## Extra Credit <span style="color: deepskyblue;">(10 points) </span>

Unlike our small example dataset, many machine learning problems must contend with large-scale data that cannot be efficiently handled by learning methods. Often, it is necessary to approximate the predictions of these learners. One common method for approximating the results of a complex learner is to compute  random Fourier features of the input data, essentially decomposing what might be a very complex dataset into a summation of simple sinusoidal functions which may make computations for the machine learning easier (for more information on Fourier transforms/series see http://www.thefouriertransform.com/). 

**Instead of expanding our data using polynomial features, try using Fourier features, i.e.,**


In [ ]:
XtrF = np.zeros( (Xtr.shape[0],5) ) # create Mx5 array to store features
XtrF[:,0] = Xtr[:,0] # place original "x" feature as X1
XtrF[:,1] = np.sin(Xtr[:,0]/2.) # place "sin(x)" feature as X2 (approx. scaled to X's range)
XtrF[:,2] = np.cos(Xtr[:,0]/2.) # place "cos(x)" feature as X3
XtrF[:,3] = np.sin(Xtr[:,0]*2./2.) # place "sin(2*x)" feature as X3
XtrF[:,4] = np.cos(Xtr[:,0]*2./2.) # place "cos(2*x)" feature as X4
# Now, XtrF has five features about each data point: "x" and four Fourier features

**Try expanding the number of Fourier features and plot the training and validation curves for this feature set. Plot your results, and discuss.**

In [ ]:
## TODO: Linear Regression with random Fourier features




<hr>

# Part 2: Cross-validation (40 points)

In the previous problem, you decided what degree of polynomial fit to use based on performance on some test data. Now suppose that you do not have access to the target values of the test data you held out in the previous problem and want to decide on the best polynomial degree.
One option would be to further split `Xtrain` into training and validation datasets and then assess performance on the validation data to choose the degree. But when training is reasonably efficient (or you have significant computational resources), it can be more effective to use cross-validation to estimate the optimal degree. Cross- validation works by creating many training/validation splits, called folds, and using all of these splits to assess the
“out-of-sample” (validation) performance by averaging them. You can do a 5-fold validation test, for example, by **(NOTE: this code is incomplete and won't run as is--you will be filling in the rest when you write your `totalMSE` function below.)**:

In [ ]:
nFolds = 5;
for iFold in range(nFolds):
    Xti,Xvi,Yti,Yvi = ml.crossValidate(Xtr,Ytr,nFolds,iFold) # use ith block as validation
    learner = ml.linear.linearRegress(... # TODO: train on Xti, Yti, the data for this fold
    J[iFold] = ... # TODO: now compute the MSE on Xvi, Yvi and save it 
# the overall estimated validation error is the average of the error on each fold 
print np.mean(J)

**Using this technique on your training data from the previous problem, find the 5-fold cross-validation MSE of linear regression at the same degrees as before, d = 1, 3, 5, 7, 10, 15, 18.**

**(a) To make your code more readable, write a function `totalMSE` that takes the degree and number of folds as arguments, and returns the cross-validation error. <span style="color: deepskyblue;">(5 points) </span>**

**(b) Plot the five-fold cross-validation error (with semilogy , as before) as a function of degree. <span style="color: deepskyblue;">(10 points)</span>**

In [ ]:
# TODO: totalMSE(d, n) to assess a model of degree d using n-fold cross-validation



# TODO: Compute and plot the five-fold cross-validation error for d = 1, 3, 5, 7, 10, 15, 18



**(c) How do the MSE estimates from five-fold cross-validation compare to the MSEs evaluated on the actual test
data (Part 1)? <span style="color: deepskyblue;">(5 points)</span>**






**(d) Which polynomial degree do you recommend based on five-fold cross-validation error? <span style="color: deepskyblue;">(5 points)</span>**

**(e) For the degree that you picked in step d, plot (with semilogy ) the cross-validation error as the number of folds is varied from nFolds = 2, 3, 4, 5, 6, 10, 12, 15. What pattern do you observe, and how do you explain why it occurs? <span style="color: deepskyblue;">(15 points)</span>**

In [ ]:
# TODO: Compute and plot cross-validation error for varying number of folds




<hr>

# Statement of Collaboration

It is **mandatory** to include a *Statement of Collaboration* in each submission. Include the names of everyone involved in the discussions (especially in-person ones), and what was discussed. 